In [1]:
import math
from scipy.optimize import newton

class Bond:
    def __init__(self, coupon_rate=None, ytm=None, maturity=None, par=None,
                 price=None, terms_per_year=2, init_guess=0.05):
        """
        Bond class that can be instantiated with EITHER a specified yield to maturity
        OR a specified price (but not both missing). The other will be computed.

        :param coupon_rate: Annual coupon rate in decimal form (e.g. 0.05 for 5%)
        :param ytm: Annual yield-to-maturity in decimal form
        :param maturity: Maturity in years
        :param par: Par (face) value of the bond
        :param price: Current market price of the bond
        :param terms_per_year: Number of coupon periods per year (e.g. 2 for semiannual)
        :param init_guess: Initial guess for the YTM solver
        """

        if (coupon_rate is None or par is None or maturity is None):
            raise ValueError("coupon_rate, par, and maturity must be provided.")

        self.coupon_rate = coupon_rate
        self.maturity = maturity
        self.par = par
        self.terms_per_year = terms_per_year
        self.init_guess = init_guess

        # The coupon payment per period
        self.coupon_payment = (self.coupon_rate * self.par) / self.terms_per_year

        # The number of total coupon periods
        self.total_periods = int(self.maturity * self.terms_per_year)

        # If neither ytm nor price is given, that's an error
        if ytm is None and price is None:
            raise ValueError("Either ytm or price must be provided.")

        # If YTM is provided, store it; else compute from the given price
        if ytm is not None:
            self.ytm_annual = ytm
        else:
            self.ytm_annual = self._calculate_ytm_from_price(price)

        # Now set the price accordingly if not given
        if price is None:
            self.price = self._calculate_price_from_ytm(self.ytm_annual)
        else:
            self.price = price

        # Precompute these
        self.macaulay_duration = self._calculate_macaulay_duration()
        self.modified_duration = self._calculate_modified_duration()
        self.convexity = self._calculate_convexity()

    def _calculate_ytm_from_price(self, price):
        """
        Numerically solve for annual YTM given a bond price and par/coupon details.
        """

        def ytm_equation(ytm_guess):
            """
            Returns f(ytm_guess) so that we want f(ytm_guess) = 0.
            The present value of all coupons + redemption at par - price = 0
            """
            per_rate = ytm_guess / self.terms_per_year
            # discount each coupon
            pv_coupons = sum(self.coupon_payment / (1+per_rate)**i
                             for i in range(1, self.total_periods+1))
            # discount the redemption
            pv_par = self.par / (1+per_rate)**(self.total_periods)
            return pv_coupons + pv_par - price

        # Solve via newton using initial guess
        ytm_solved = newton(ytm_equation, self.init_guess)
        return ytm_solved

    def _calculate_price_from_ytm(self, ytm_annual):
        """
        Compute the bond's price from the given annual YTM.
        """
        per_rate = ytm_annual / self.terms_per_year
        # discount each coupon
        pv_coupons = sum(self.coupon_payment / (1+per_rate)**i
                         for i in range(1, self.total_periods+1))
        # discount redemption
        pv_par = self.par / (1+per_rate)**(self.total_periods)
        return pv_coupons + pv_par

    def set_ytm(self, new_ytm):
        """
        Update the YTM, then recalc price, durations, convexity.
        """
        self.ytm_annual = new_ytm
        self.price = self._calculate_price_from_ytm(new_ytm)
        self.macaulay_duration = self._calculate_macaulay_duration()
        self.modified_duration = self._calculate_modified_duration()
        self.convexity = self._calculate_convexity()

    def _calculate_macaulay_duration(self):
        """
        Calculates Macaulay duration in *years*.
        """
        per_rate = self.ytm_annual / self.terms_per_year
        # Weighted sum of times
        numerator = 0.0
        for i in range(1, self.total_periods+1):
            t = i / self.terms_per_year  # in years
            cf = self.coupon_payment
            numerator += t * cf / (1+per_rate)**i

        # final redemption
        numerator += (self.maturity) * self.par / (1+per_rate)**(self.total_periods)

        return numerator / self.price

    def _calculate_modified_duration(self):
        """
        Modified duration = Macaulay duration / (1 + yield/periods_per_year).
        """
        per_rate = self.ytm_annual / self.terms_per_year
        return self.macaulay_duration / (1 + per_rate)

    def dv01(self, bp=1):
        """
        Compute the change in price for a 1 bp shift in YTM.
        Typically DV01 = - (partial derivative of price w.r.t. yield).
        But here we do a discrete approximation.
        """
        delta_yield = bp / 10000  # 1 basis point in decimal
        bumped_ytm = self.ytm_annual + delta_yield
        new_price = self._calculate_price_from_ytm(bumped_ytm)
        return new_price - self.price

    def _calculate_convexity(self):
        """
        Classical convexity measure in continuous compounding sense.
        Here we do discrete approximation:
          sum( t*(t+1)*CF / (1+y/term)^t+2 ) / Price
        plus final redemption.
        Then scale if needed to 'annualize'.
        """
        per_rate = self.ytm_annual / self.terms_per_year
        numerator = 0.0
        for i in range(1, self.total_periods+1):
            numerator += i*(i+1)*self.coupon_payment / (1+per_rate)**(i+2)

        numerator += self.total_periods * (self.total_periods + 1) * self.par / (1+per_rate)**(self.total_periods+2)

        return numerator / (self.price * (self.terms_per_year**2))


In [2]:
Bond1 = Bond(0.09, 0.09, 5, 100)
Bond2 = Bond(0.09, 0.09, 25, 100)
Bond3 = Bond(0.06, 0.09, 5, 100)
Bond4 = Bond(0.06, 0.09, 25, 100)
Bond5 = Bond(0, 0.09, 5, 100)
Bond6 = Bond(0, 0.09, 25, 100)
Bond_portfolio = [Bond1, Bond2, Bond3, Bond4, Bond5, Bond6]
for y in range(6, 11):
    for x in Bond_portfolio:
        x.set_ytm(y/100)
        print(f"YTM: {y}%\tcoupon: {x.coupon_rate}\tMaturity:{x.maturity}  \tDV01: {x.dv01():.4f}\tPrice: {x.price:.4f}")
    print()

YTM: 6%	coupon: 0.09	Maturity:5  	DV01: -0.0459	Price: 112.7953
YTM: 6%	coupon: 0.09	Maturity:25  	DV01: -0.1651	Price: 138.5946
YTM: 6%	coupon: 0.06	Maturity:5  	DV01: -0.0426	Price: 100.0000
YTM: 6%	coupon: 0.06	Maturity:25  	DV01: -0.1285	Price: 100.0000
YTM: 6%	coupon: 0	Maturity:5  	DV01: -0.0361	Price: 74.4094
YTM: 6%	coupon: 0	Maturity:25  	DV01: -0.0553	Price: 22.8107

YTM: 7%	coupon: 0.09	Maturity:5  	DV01: -0.0437	Price: 108.3166
YTM: 7%	coupon: 0.09	Maturity:25  	DV01: -0.1383	Price: 123.4556
YTM: 7%	coupon: 0.06	Maturity:5  	DV01: -0.0405	Price: 95.8417
YTM: 7%	coupon: 0.06	Maturity:25  	DV01: -0.1066	Price: 88.2722
YTM: 7%	coupon: 0	Maturity:5  	DV01: -0.0342	Price: 70.8919
YTM: 7%	coupon: 0	Maturity:25  	DV01: -0.0432	Price: 17.9053

YTM: 8%	coupon: 0.09	Maturity:5  	DV01: -0.0416	Price: 104.0554
YTM: 8%	coupon: 0.09	Maturity:25  	DV01: -0.1165	Price: 110.7411
YTM: 8%	coupon: 0.06	Maturity:5  	DV01: -0.0385	Price: 91.8891
YTM: 8%	coupon: 0.06	Maturity:25  	DV01: -0.0889	P

In [3]:
ChangeInBP = [-300, -200, -100, -50, -10, -1, 1, 10, 50, 100, 200, 300]
Bond1 = Bond(0.09, 0.09, 5, 100)
Bond2 = Bond(0.09, 0.09, 25, 100)
Bond3 = Bond(0.06, 0.09, 5, 100)
Bond4 = Bond(0.06, 0.09, 25, 100)
Bond5 = Bond(0, 0.09, 5, 100)
Bond6 = Bond(0, 0.09, 25, 100)
Bond_portfolio = [Bond1, Bond2, Bond3, Bond4, Bond5, Bond6]
for bp in ChangeInBP:
    for x in Bond_portfolio:
        print(f"change in bp: {bp}\tcoupon: {x.coupon_rate}\tMaturity:{x.maturity}  \tDV01: {x.dv01(bp = bp):.4f} \tpercentage change in price: {x.dv01(bp = bp)/x.price*100 :.2f}%")
    print()

change in bp: -300	coupon: 0.09	Maturity:5  	DV01: 12.7953 	percentage change in price: 12.80%
change in bp: -300	coupon: 0.09	Maturity:25  	DV01: 38.5946 	percentage change in price: 38.59%
change in bp: -300	coupon: 0.06	Maturity:5  	DV01: 11.8691 	percentage change in price: 13.47%
change in bp: -300	coupon: 0.06	Maturity:25  	DV01: 29.6430 	percentage change in price: 42.13%
change in bp: -300	coupon: 0	Maturity:5  	DV01: 10.0166 	percentage change in price: 15.56%
change in bp: -300	coupon: 0	Maturity:25  	DV01: 11.7397 	percentage change in price: 106.04%

change in bp: -200	coupon: 0.09	Maturity:5  	DV01: 8.3166 	percentage change in price: 8.32%
change in bp: -200	coupon: 0.09	Maturity:25  	DV01: 23.4556 	percentage change in price: 23.46%
change in bp: -200	coupon: 0.06	Maturity:5  	DV01: 7.7108 	percentage change in price: 8.75%
change in bp: -200	coupon: 0.06	Maturity:25  	DV01: 17.9152 	percentage change in price: 25.46%
change in bp: -200	coupon: 0	Maturity:5  	DV01: 6.499

In [5]:
Bond1 = Bond(0.09, 0.09, 5, 100)
Bond2 = Bond(0.09, 0.09, 25, 100)
Bond3 = Bond(0.06, 0.09, 5, 100)
Bond4 = Bond(0.06, 0.09, 25, 100)
Bond5 = Bond(0, 0.09, 5, 100)
Bond6 = Bond(0, 0.09, 25, 100)
Bond_portfolio = [Bond1, Bond2, Bond3, Bond4, Bond5, Bond6]
for b in Bond_portfolio:
    print(f'{b.maturity}-year  \t{b.coupon_rate*100}%  \tInitial Price: {b.price:.4f}  \tnew price: {b._calculate_price_from_ytm(0.0901):.4f} DV01:{b.dv01():.4f}')

5-year  	9.0%  	Initial Price: 100.0000  	new price: 99.9604 DV01:-0.0396
25-year  	9.0%  	Initial Price: 100.0000  	new price: 99.9013 DV01:-0.0987
5-year  	6.0%  	Initial Price: 88.1309  	new price: 88.0943 DV01:-0.0366
25-year  	6.0%  	Initial Price: 70.3570  	new price: 70.2824 DV01:-0.0746
5-year  	0%  	Initial Price: 64.3928  	new price: 64.3620 DV01:-0.0308
25-year  	0%  	Initial Price: 11.0710  	new price: 11.0445 DV01:-0.0265


In [7]:
Bond4p5 = Bond(coupon_rate=0.09, ytm=0.09, maturity=5, par=100)
print(Bond4p5.macaulay_duration)
print(Bond4p5.modified_duration)

4.134395247540063
3.956359088555084


In [8]:
Bond4p6 = Bond(coupon_rate=0.06, ytm=0.09, maturity=5, par=100)
print(Bond4p6.macaulay_duration)
print(Bond4p6.modified_duration)

4.345212988734008
4.158098553812448


In [9]:
Bond1 = Bond(0.09, 0.09, 5, 100)
Bond2 = Bond(0.09, 0.09, 25, 100)
Bond3 = Bond(0.06, 0.09, 5, 100)
Bond4 = Bond(0.06, 0.09, 25, 100)
Bond5 = Bond(0, 0.09, 5, 100)
Bond6 = Bond(0, 0.09, 25, 100)
Bond_portfolio = [Bond1, Bond2, Bond3, Bond4, Bond5, Bond6]
for b in Bond_portfolio:
    print(f'{b.maturity}-year  \t{b.coupon_rate*100}%  \tMacaulay Duration: {b.macaulay_duration:.4f}  \tModified Duration: {b.modified_duration:.4f}')

5-year  	9.0%  	Macaulay Duration: 4.1344  	Modified Duration: 3.9564
25-year  	9.0%  	Macaulay Duration: 10.3256  	Modified Duration: 9.8810
5-year  	6.0%  	Macaulay Duration: 4.3452  	Modified Duration: 4.1581
25-year  	6.0%  	Macaulay Duration: 11.0953  	Modified Duration: 10.6175
5-year  	0%  	Macaulay Duration: 5.0000  	Modified Duration: 4.7847
25-year  	0%  	Macaulay Duration: 25.0000  	Modified Duration: 23.9234


In [11]:
y_range = range(7, 15)
Bond25y9 = Bond(coupon_rate=0.09, ytm=0.09, maturity=25, par=100)
for y in y_range:
    Bond25y9.set_ytm(y/100)
    print(f'YTM: {y}% \tModified Duration: {Bond25y9.modified_duration:.2f}')

YTM: 7% 	Modified Duration: 11.21
YTM: 8% 	Modified Duration: 10.53
YTM: 9% 	Modified Duration: 9.88
YTM: 10% 	Modified Duration: 9.27
YTM: 11% 	Modified Duration: 8.70
YTM: 12% 	Modified Duration: 8.16
YTM: 13% 	Modified Duration: 7.66
YTM: 14% 	Modified Duration: 7.21


In [12]:
q2BondA = Bond(0.08, 0.08, 2, 100, 100)
q2BondB = Bond(0.09, 0.08, 2, 100, 100)

print(q2BondA.dv01())
print(q2BondB.dv01())

-0.018147337657353546
1.7965864280812553


In [13]:
Bond4p14 = Bond(coupon_rate=0.09, ytm=0.09, maturity=5, par=100)
print(Bond4p14._calculate_convexity())
Bond4p15 = Bond(coupon_rate=0.06, ytm=0.09, maturity=5, par=100)
print(Bond4p15._calculate_convexity())

19.452564325142994
20.848106137219354


In [14]:
Bond1 = Bond(0.09, 0.09, 5, 100)
Bond2 = Bond(0.09, 0.09, 25, 100)
Bond3 = Bond(0.06, 0.09, 5, 100)
Bond4 = Bond(0.06, 0.09, 25, 100)
Bond5 = Bond(0, 0.09, 5, 100)
Bond6 = Bond(0, 0.09, 25, 100)
Bond_portfolio = [Bond1, Bond2, Bond3, Bond4, Bond5, Bond6]
for b in Bond_portfolio:
    print(f'{b.maturity}-year  \t{b.coupon_rate*100}%  \tprice: {b.price:.4f}  \tConvexity: {b._calculate_convexity():.4f}')

5-year  	9.0%  	price: 100.0000  	Convexity: 19.4526
25-year  	9.0%  	price: 100.0000  	Convexity: 160.7211
5-year  	6.0%  	price: 88.1309  	Convexity: 20.8481
25-year  	6.0%  	price: 70.3570  	Convexity: 182.9110
5-year  	0%  	price: 64.3928  	Convexity: 25.1826
25-year  	0%  	price: 11.0710  	Convexity: 583.7778
